# Chat-SGP Tutorial

This notebook provides a comprehensive tutorial on using the Chat-SGP system for renewable energy optimization.

## What is Chat-SGP?

Chat-SGP is a multi-agent system that helps you analyze renewable energy scenarios by:
1. **CoderAgent**: Parses natural language questions into optimization modifications
2. **OptimizerAgent**: Solves the energy optimization problem
3. **InterpreterAgent**: Provides human-readable explanations of results

Let's get started!


In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

from chatsgp.agents.coder_agent import CoderAgent
from chatsgp.agents.optimizer_agent import OptimizerAgent
from chatsgp.agents.interpreter_agent import InterpreterAgent
from chatsgp.agents.orchestrator import Orchestrator
from chatsgp.utils.llm_backend import LLM
from chatsgp.config import get_config
from chatsgp.utils.visualization import (
    plot_energy_flows,
    plot_cost_comparison,
    plot_24h_profile,
    plot_multiple_scenarios
)
import json
import matplotlib.pyplot as plt

print("✓ All imports successful!")


## Step 1: Setup and Configuration

First, let's load ICL examples and initialize the agents.


In [ ]:
# Load ICL examples
def load_icl(path='chatsgp/icl/examples.jsonl'):
    """Load ICL examples for the CoderAgent"""
    p = Path(path)
    ex = []
    if p.exists():
        for line in p.read_text(encoding='utf-8').splitlines():
            ex.append(json.loads(line))
    return ex

# Initialize agents
icl_examples = load_icl()
llm = LLM()  # Will use OpenAI API if available, otherwise None
coder = CoderAgent(icl_examples, llm=llm)
optimizer = OptimizerAgent()
interpreter = InterpreterAgent()

# Create orchestrator
orchestrator = Orchestrator(coder, optimizer, interpreter)

print("✓ Agents initialized successfully!")
if llm.client:
    print("✓ LLM client available (using OpenAI API)")
else:
    print("⚠ LLM client not available (will use rule-based parsing)")


## Step 2: Basic Usage

Let's start with a simple question about PV generation.


In [ ]:
# Example 1: PV generation increase
question1 = "What happens if PV generation increases by 20%?"

print(f"Question: {question1}\n")
result1 = orchestrator.run_question(question1, solver='pulp')

print(f"Status: {result1['result']['status']}")
print(f"Total Cost: EUR {result1['result']['objective']:.2f}")
print(f"\nAnswer:\n{result1['answer']}")


## Step 3: Visualizing Results

Let's visualize the energy flows and see what happened.


In [ ]:
# Get the data from the optimizer
# We need to run the optimizer again to get the full data
ops = result1['ops']
data, res = optimizer.run(ops, solver='pulp')

# Plot energy flows
plot_energy_flows(data, res, show=True)


## Step 4: Comparing Scenarios

Let's compare the baseline with a modified scenario to see the cost difference.


In [ ]:
# Get baseline (no modifications)
baseline_ops = {'ops': [], 'explanation': 'baseline'}
baseline_data, baseline_res = optimizer.run(baseline_ops, solver='pulp')
baseline_cost = baseline_res['objective']

# Get scenario cost
scenario_cost = result1['result']['objective']

# Plot cost comparison
plot_cost_comparison(
    baseline_cost, 
    scenario_cost,
    scenario_name="PV +20%",
    show=True
)

print(f"Baseline Cost: EUR {baseline_cost:.2f}")
print(f"Scenario Cost: EUR {scenario_cost:.2f}")
print(f"Difference: EUR {scenario_cost - baseline_cost:.2f}")


In [ ]:
# Example 2: Import price increase
question2 = "What happens if imports increase by 10%?"
print(f"Question: {question2}\n")
result2 = orchestrator.run_question(question2, solver='pulp')
print(f"Cost: EUR {result2['result']['objective']:.2f}")
print(f"\nAnswer:\n{result2['answer']}\n")

# Example 3: Load shifting
question3 = "What if we shift 25% of load from hour 13 to hour 14?"
print(f"Question: {question3}\n")
result3 = orchestrator.run_question(question3, solver='pulp')
print(f"Cost: EUR {result3['result']['objective']:.2f}")
print(f"\nAnswer:\n{result3['answer']}\n")


## Step 6: Multiple Scenarios Comparison

Let's compare multiple scenarios side by side.


In [ ]:
# Run multiple scenarios
scenarios = [
    {'name': 'Baseline', 'cost': baseline_cost},
    {'name': 'PV +20%', 'cost': result1['result']['objective']},
    {'name': 'Import +10%', 'cost': result2['result']['objective']},
    {'name': 'Load Shift', 'cost': result3['result']['objective']},
]

# Plot comparison
plot_multiple_scenarios(scenarios, show=True)


## Step 7: Customizing Parameters

You can customize system parameters using a configuration file.


In [ ]:
# Load configuration
config = get_config()

# Display current configuration
print("Current Configuration:")
print(f"  Battery Capacity: {config.get('battery.capacity_kwh')} kWh")
print(f"  Battery Efficiency: {config.get('battery.efficiency')}")
print(f"  Import Price: EUR {config.get('prices.import')}/kWh")
print(f"  Export Price: EUR {config.get('prices.export')}/kWh")
print(f"  Default Solver: {config.get('optimization.default_solver')}")

# You can create a config.yaml file to customize these values
# See config.yaml.example for the format


## Step 8: Visualizing Profiles

Let's visualize the PV and Load profiles.


In [ ]:
# Get default data
default_data, _ = optimizer.run({'ops': [], 'explanation': 'baseline'}, solver='pulp')

# Plot PV profile
plot_24h_profile(default_data['PV'], 'PV Generation', show=True)

# Plot Load profile
plot_24h_profile(default_data['Load'], 'Load Demand', show=True)


## Next Steps

- Try different questions and scenarios
- Customize parameters in `config.yaml`
- Explore the example scenarios in `examples/scenarios/`
- Check the README.md for more information
- Use `--debug` flag in CLI to see detailed processing

Happy optimizing! 🚀
